# CITS4012 Project2

Theo Andily  22764884

Hanlin Zhang 22541459

## Modules

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

import numpy as np
import matplotlib.pyplot as plt
import gensim.downloader as api

from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet

import gensim
import re
from gensim.corpora import Dictionary

import nltk
from nltk.corpus import stopwords

import timeit
from collections import Counter
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import en_core_web_sm
import os

from argparse import Namespace
import collections
from torch.utils.data import Dataset, DataLoader
import json
from tqdm.notebook import tqdm

## Data pre-processing and preparation

### Setting up

In [55]:
args = Namespace(
    # Data and Path hyper parameters
    seed=1337,
    output="output.csv",
    train_proportion=0.7,
    val_proportion=0.1,
    test_proportion=0.2,
    domain_model_dir="model_storage/domain_model/",
    domain_file_name="domain.model",
    expand_filepaths_to_save_dir=True
) 

### Read data

In [56]:
import pandas as pd

def load_csv(csv_file):
    return pd.read_csv(csv_file)

job_data = load_csv("seek_australia.csv")

### Clean data

In [57]:
# try to remove the broken characters

for each in ['š', 'ª', 'º', 'Ÿ', 'µ', 'ˆ', 'œ', 'é', 'è', 'ï', 'Š', 'â', 'ç', 'æ', 'å', 'ä', 'ð', 
            'Ã', 'Â', 'Å', 'Ä', '®', '€', '™', 'ž', 'œ', '¼', '½', '¾', '¹', '²','³', '\xa0']:
    try:
        job_data['job_description_clean'] = job_data['job_description_clean'].replace({each:''}, regex=True)
    except:
        job_data['job_description_clean'] = job_data['job_description'].replace({each:''}, regex=True)

job_data['job_description_clean'] = job_data['job_description_clean'].str.replace(r'[^\w\s]+', '')

job_data['job_type'] = np.where(job_data['job_type'] == 'Full Time', 'Full Time', 'Other')  # replace the job type to be Other except Full Time

job_data = job_data[job_data['job_description_clean'].notna()]  # keep the job data sample which is not na only

C:\Users\theor\AppData\Local\Temp\ipykernel_7976\154546671.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  job_data['job_description_clean'] = job_data['job_description_clean'].str.replace(r'[^\w\s]+', '')


Here we removed unnessaary unicode characters, punctuation and change the job type to either be full time or other. The way we remove punctuation could join two or more words together and therefore loses its sematic meaning.

## Data Exploration

### Get job description from the data set and get random 2000 samples

Here we calculate the tfidf of each category which runs over all the job description and hence takes a while to run.

In [58]:
def tfidf_pipe(original_text: list):
    stopword = stopwords.words('english')
    """ This function returns the featured words in after tfidf up to 500 words """
    if len(original_text) > 1 and original_text != np.nan:
        tfidf_vectorizer = TfidfVectorizer(smooth_idf=True)
        tfidf = tfidf_vectorizer.fit_transform(original_text)
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_values = np.argsort(tfidf.toarray()).flatten()[::-1]
        tfidf_return = []
        counter = 0
        while len(tfidf_return) < 10:
            if feature_names[tfidf_values].tolist()[counter] not in stopword and feature_names[tfidf_values].tolist()[counter].isdigit() != True:
                tfidf_return.append(feature_names[tfidf_values].tolist()[counter])
            counter += 1
        return tfidf_return
    return ''

In [59]:
job_description_per_category = {}
tfidf_job_category = {}
unique_category=job_data['category'].unique()
for jobs in unique_category:
    start = timeit.default_timer()
    tfidf_values = tfidf_pipe(job_data.loc[job_data['category'] == jobs]['job_description_clean'].tolist())
    stop = timeit.default_timer()
    tfidf_job_category[jobs] = tfidf_values
    print('Completed tfidf for', jobs, 'time taken: ', str(round((stop - start), 3)) +'s')

Completed tfidf for Retail & Consumer Products time taken:  6.132s
Completed tfidf for Government & Defence time taken:  11.673s
Completed tfidf for Trades & Services time taken:  40.048s
Completed tfidf for Manufacturing, Transport & Logistics time taken:  21.966s
Completed tfidf for Sales time taken:  12.719s
Completed tfidf for Community Services & Development time taken:  5.539s
Completed tfidf for Healthcare & Medical time taken:  31.033s
Completed tfidf for Information & Communication Technology time taken:  33.995s
Completed tfidf for Mining, Resources & Energy time taken:  4.935s
Completed tfidf for Construction time taken:  9.654s
Completed tfidf for Design & Architecture time taken:  0.899s
Completed tfidf for Call Centre & Customer Service time taken:  3.614s
Completed tfidf for Marketing & Communications time taken:  4.119s
Completed tfidf for Administration & Office Support time taken:  14.234s
Completed tfidf for Banking & Financial Services time taken:  3.373s
Completed 

In [60]:
job_data.head()
job_data['tfidf'] = ''
for jobs in unique_category:
    job_data.loc[job_data['category'] == jobs, 'tfidf'] = [' '.join(tfidf_job_category[jobs])] * len(job_data.loc[job_data['category'] == jobs])

In [61]:
cleaned_data = job_data[['job_description_clean', 'job_type', 'category', 'tfidf']] # keep column with name job_description_clean, job_type, category and tfidf

Above we remove the useless columns (retain job_description, job_type, category and tfidf top 10 words).

In [62]:
cleaned_data.to_csv('cleaned_data_new.csv', index=False)    # store the result into the file

In [63]:
# Splitting train by category
# Create dict
by_category = collections.defaultdict(list)
for _, row in cleaned_data.iterrows():
    by_category[row.job_type].append(row.to_dict())

In [64]:
# Create split data
final_list = []
np.random.seed(args.seed)
for _, item_list in sorted(by_category.items()):
    np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_proportion*n)
    n_val = int(args.val_proportion*n)
    n_test = int(args.test_proportion*n)
    
    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  
    
    # Add to final list
    final_list.extend(item_list)

In [65]:
# Write split data to file
final_data = pd.DataFrame(final_list)

In [66]:
final_data.split.value_counts()

train    20758
test      5932
val       2965
Name: split, dtype: int64

In [67]:
# Clean sentences
def preprocess_text(text):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_data.job_description_clean = final_data.job_description_clean.apply(preprocess_text)

In [68]:
# have a quick look how the data frame looks like
final_data.head()

,job_description_clean,job_type,category,tfidf,split
0,ph super uncapped commission overtime availab...,Full Time,Sales,startrack collate freight analyst post unrival...,train
1,universal recruitment has years of experience...,Full Time,"Manufacturing, Transport & Logistics",controller malec bros fleet multitasking movem...,train
2,goodlife health clubsis the largest australia...,Full Time,Call Centre & Customer Service,service desk consultant customer incidents tec...,train
3,the company our client has a very dynamic and...,Full Time,Call Centre & Customer Service,service desk consultant customer incidents tec...,train
4,about the business we are a small family busi...,Full Time,Trades & Services,pest anticimex flick control australias hygien...,train


In [69]:
# Write munged data to CSV
final_data.to_csv(args.output, index=False)

### TSNE

In [70]:
def interactive_tsne(text_labels, tsne_array):
    '''makes an interactive scatter plot with text labels for each point'''

    # Define a dataframe to be used by bokeh context
    bokeh_df = pd.DataFrame(tsne_array, text_labels, columns=['x','y'])
    bokeh_df['text_labels'] = bokeh_df.index

    # interactive controls to include to the plot
    TOOLS="hover, zoom_in, zoom_out, box_zoom, undo, redo, reset, box_select"

    p = figure(tools=TOOLS, plot_width=700, plot_height=700)

    # define data source for the plot
    source = ColumnDataSource(bokeh_df)

    # scatter plot
    p.scatter('x', 'y', source=source, fill_alpha=0.6,
              fill_color="#8724B5",
              line_color=None)

    # text labels
    labels = LabelSet(x='x', y='y', text='text_labels', y_offset=8,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')

    p.add_layout(labels)

    # show plot inline
    output_notebook()
    show(p)

In [71]:
def convert(tup, dict):
    for a, b in tup:
        dict[a] = b
    return dict

Here we merge all the job description data from the sample in train data set into one str, then do some simple preprocess.

In [72]:
train_df = final_data[final_data['split'] == 'train']
job_descript_str = ' '.join(train_df['job_description_clean'])
doc_tokenized = gensim.utils.simple_preprocess(str(job_descript_str), deacc=True)

In [73]:
dictionary = Dictionary()
BoW_corpus = dictionary.doc2bow(doc_tokenized, allow_update=True)
BoW_corpus = [(dictionary[id], freq) for id, freq in BoW_corpus]

BoW_corpus_dict = dict()
convert_result = convert(BoW_corpus, BoW_corpus_dict)  # Here is a varible placement just to remove the cell from printing the results

Here we build and train the domain specific embeddings from the word2vec library with the bag of word frequency.

In [74]:
cores = 16
model = Word2Vec(min_count=3,
                     window=2,
                     vector_size=100,
                     sample=6e-5, 
                     alpha=0.05, 
                     min_alpha=0.0005, 
                     negative=15,
                     workers=cores-1)

In [75]:
from time import time

t = time()

model.build_vocab([doc_tokenized], progress_per=10)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.02 mins


In [76]:
if args.expand_filepaths_to_save_dir:
    args.domain_model = os.path.join(args.domain_model_dir,
                                        args.domain_file_name)

In [77]:
# If model exists, read it. Otherwise train it.
if os.path.exists(args.domain_model_dir):
    model = Word2Vec.load(args.domain_model)
else:
    t = time()
    model.train(doc_tokenized, total_examples=model.corpus_count, epochs=100, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
    os.makedirs(args.domain_model_dir)
    model.save(args.domain_model)

For a better visualization, it is better to plot some of words, instead of all the words.

In [78]:
vocab_sorted = dict(sorted(BoW_corpus_dict.items(), key=lambda item: item[1], reverse=True))

stopwords_list = stopwords.words('english')
input_vocab = []
input_vocab =  [word for word in vocab_sorted if word in model.wv.key_to_index.keys() and word not in stopwords_list]
points = len(input_vocab)
X = model.wv[input_vocab]
X_tsne = TSNE(n_components=2, random_state=4012).fit_transform(X[:70])

interactive_tsne(list(input_vocab)[:70], X_tsne)

c:\envs\cits4012\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\envs\cits4012\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Loading BokehJS ...

We plotted the first 70 words in our input vocab from the Word2Vec model that we trained and plotted them using the TSNE plotter. The plot above shows the similarity of the words by their distance from neighbouring words. For example, the word provide is simialr to training, care, knowledge, customer key and management based on the word vectors but they are not a good representation since the distance between excellent is far from great and successful. This is due to the domain speicific nature of the word vectoriser since it was only given the job description as training data.

## Task 1

### FNN model using the top 10 tfidf words

This section we build a feed forwards neural network using the top 10 tfidf words with one hidden layer and we followed closely to the codes on the labs and modified to our need for tfidf. The fnn only used the onehotencoder vectoriser which is implemented from the lab and the use of onehotencoder is not very efficient given their nature of high feature size as it builds one vector per unique word. The training time will depends on the number of epochs but we kept a 100 epochs for all of the models including in task 2, it should take a few minutes for ffnn, and cnn using tfidf but it will take a few hours for cnn with job description.

In [79]:
from argparse import Namespace
args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='model.pth',
    tfidf_csv='output.csv',
    save_dir='model_storage/fnn',
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.0001,
    num_epochs=100,
    seed=1337,
    # Runtime options
    cuda=True,
    device='cuda',
)

In [80]:
class tfidfDataset(Dataset):
    def __init__(self, tfidf_df, vectorizer):
        """
        Args:
        tfidf_df (pandas.DataFrame): the dataset
        vectorizer (tfidfVectorizer): vectorizer instantiated from dataset
        """
        self.tfidf_df = tfidf_df
        self._vectorizer = vectorizer
        self.train_df = self.tfidf_df[self.tfidf_df.split=='train']
        self.train_size = len(self.train_df)
        self.val_df = self.tfidf_df[self.tfidf_df.split=='val']
        self.validation_size = len(self.val_df)
        self.test_df = self.tfidf_df[self.tfidf_df.split=='test']
        self.test_size = len(self.test_df)
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.validation_size),
                            'test': (self.test_df, self.test_size)}
        self.set_split('train')
    @classmethod
    def load_dataset_and_make_vectorizer(cls, tfidf_csv):
        """Load dataset and make a new vectorizer from scratch
        Args:
            tfidf_csv (str): location of the dataset
        Returns:
            an instance of tfidfDataset
        """
        tfidf_df = pd.read_csv(tfidf_csv)
        return cls(tfidf_df, tfidfVectorizer.from_dataframe(tfidf_df))
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer
    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    def __len__(self):
        return self._target_size
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        tfidf_vector = \
        self._vectorizer.vectorize(row.tfidf)
        job_type_index = \
        self._vectorizer.job_type_vocab.lookup_token(row.job_type)
        return {'x_data': tfidf_vector,
                'y_target': job_type_index}
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
        batch_size (int)
        Returns:
        number of batches in the dataset
        """
        return len(self) // batch_size

In [81]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

In [82]:
class tfidfVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, tfidf_vocab, job_type_vocab):
        """
        Args:
            tfidf_vocab (Vocabulary): maps words to integers
            job_type_vocab (Vocabulary): maps type labels to integers
        """
        self.tfidf_vocab = tfidf_vocab
        self.job_type_vocab = job_type_vocab
    def vectorize(self, tfidf):
        """Create a collapsed one hot vector for the tfidf
        Args:
            tfidf (str): the tfidf
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.tfidf_vocab), dtype=np.float32)
        for token in tfidf.split(" "):
            if token not in string.punctuation:
                one_hot[self.tfidf_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, tfidf_df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            tfidf_df (pandas.DataFrame): the tfidf dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the tfidfVectorizer
        """
        tfidf_vocab = Vocabulary(add_unk=True)
        job_type_vocab = Vocabulary(add_unk=False)
        # Add job_types
        for job_type in sorted(set(tfidf_df.job_type)):
            job_type_vocab.add_token(job_type)
        # Add top words if count > provided count
        word_counts = Counter()
        for tfidf in tfidf_df.tfidf:
            for word in tfidf.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                tfidf_vocab.add_token(word)
        return cls(tfidf_vocab, job_type_vocab)
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a tfidfVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the tfidfVectorizer class
        """
        tfidf_vocab = Vocabulary.from_serializable(contents['tfidf_vocab'])
        job_type_vocab = Vocabulary.from_serializable(contents['job_type_vocab'])
        return cls(tfidf_vocab=tfidf_vocab, job_type_vocab=job_type_vocab)
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'tfidf_vocab': self.tfidf_vocab.to_serializable(),
                'job_type_vocab': self.job_type_vocab.to_serializable()}

In [83]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
    ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, 
                            shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [84]:
class tfidfClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(tfidfClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

In [85]:
def make_train_state(args):
    return {'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': 1,
            'test_acc': 1}

train_state = make_train_state(args)
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

# dataset and vectorizer
dataset = tfidfDataset.load_dataset_and_make_vectorizer(args.tfidf_csv)
vectorizer = dataset.get_vectorizer()
# model
classifier = tfidfClassifier(num_features=len(vectorizer.tfidf_vocab))
classifier = classifier.to(args.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

In [86]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [87]:
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

In [88]:
dataset.set_split('test')
batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()
for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index + 1)
    # compute the accuracy
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index + 1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [89]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.596
Test Accuracy: 67.37


We tried various parameters to find the best accuracy with the following parameters with three run time and average them, though given the nature of ffnn there will be different result but they will be close to the average.

| batch_size, learning_rate, early_stopping_criteria | loss  | Accuracy |
|----------------------------------------------------|-------|----------|
| 128, 0.001, 5                                      | 0.596 | 67.19    |
| 128, 0.01, 5                                       | 0.601 | 65.96    |
| 64, 0.01, 5                                        | 0.608 | 65.15    |
| 128, 0.0001, 5                                     | 0.597 | 67.31    |
| 64, 0.0001, 5                                      | 0.598 | 67.24    |
| 128, 0.0001, 8                                     | 0.597 | 67.27    |
| 128, 0.0001, 3                                     | 0.597 | 67.24    |

From the table above the biggest factor affecting the accuracy is the learning rate and the best set of parameters is with batch_size=128, learning_rate=0.0001, early_stopping_criteria=5. Though we did not consider adding more layers which could increase the perfomance.

### CNN Conv1d based model using the top 10 tfidf words

The main objective of this section is to build a cnn model to classify the job type binary classification using the tfidf columns which was made in the data preprocessing and since the most of the tfidf values are similar per rows as they have the same category, this makes it harder for the model to classify the job types and should not give a higher probability per prediciton.

To swap between the word embedding set the different args in the following cells:

onehotencoder: use_glove=False, use_domain_specific=False,

pretrained: use_glove=True, use_domain_specific=False

domain_specific: use_glove=False, use_domain_specific=True

The models are saved in the specified folder which are shown in the cell below.

In [90]:
args = Namespace(
    # Data and Path hyper parameters
    data_set="cleaned_data_new.csv",
    vectorizer_file="cnn_vectorizer.json",
    model_state_file="cnn_model.pth",
    output="output.csv",
    save_dir_10_word_none_embedding="model_storage\cnn_10_word_none_embedding",
    save_dir_10_words_pretrained="model_storage\cnn_10_word_pretrained",
    save_dir_10_words_domain_speicific="model_storage\cnn_10_word_domain_specific",
    domain_model_dir="model_storage\domain_model\domain.model",
    domain_file_name="domain.model",
    # Model hyper parameters
    glove_filepath='glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    use_domain_specific=True,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    kernel_size=3,
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=100, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    train_proportion=0.7,
    val_proportion=0.1,
    test_proportion=0.2
)  

In [91]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [92]:
if args.use_glove == True:
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir_10_words_pretrained,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir_10_words_pretrained,
                                            args.model_state_file)

        # Check CUDA
        if not torch.cuda.is_available():
            args.cuda = False

        args.device = torch.device("cuda" if args.cuda else "cpu")
            
        print("Using CUDA: {}".format(args.cuda))


        # Set seed for reproducibility
        set_seed_everywhere(args.seed, args.cuda)

        # handle dirs
        handle_dirs(args.save_dir_10_words_pretrained)

if args.use_domain_specific == True:
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir_10_words_domain_speicific,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir_10_words_domain_speicific,
                                            args.model_state_file)

        # Check CUDA
        if not torch.cuda.is_available():
            args.cuda = False

        args.device = torch.device("cuda" if args.cuda else "cpu")
            
        print("Using CUDA: {}".format(args.cuda))


        # Set seed for reproducibility
        set_seed_everywhere(args.seed, args.cuda)

        # handle dirs
        handle_dirs(args.save_dir_10_words_domain_speicific)

if args.use_glove == False and args.use_domain_specific == False:
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir_10_word_none_embedding,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir_10_word_none_embedding,
                                            args.model_state_file)

        # Check CUDA
        if not torch.cuda.is_available():
            args.cuda = False

        args.device = torch.device("cuda" if args.cuda else "cpu")
            
        print("Using CUDA: {}".format(args.cuda))


        # Set seed for reproducibility
        set_seed_everywhere(args.seed, args.cuda)

        # handle dirs
        handle_dirs(args.save_dir_10_word_none_embedding)

Using CUDA: True


In [93]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [94]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [95]:
class CNNClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, 
                 hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings default is None 
            padding_idx (int): an index representing a null position
        """
        super(CNNClassifier, self).__init__()

        if pretrained_embeddings is None:
            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
        
            
        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels=embedding_size, 
                   out_channels=num_channels, kernel_size=args.kernel_size),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=args.kernel_size, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=args.kernel_size),
            nn.ELU()
        )

        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(num_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
            x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        
        # embed and permute so features are channels
        x_embedded = self.emb(x_in).permute(0, 2, 1)

        features = self.convnet(x_embedded)

        # average and remove the extra dimension
        remaining_size = features.size(dim=2)
        features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
        features = F.dropout(features, p=self._dropout_p)
        
        # mlp classifier
        intermediate_vector = F.relu(F.dropout(self.fc1(features), p=self._dropout_p))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

In [96]:
class CNNTfidfVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, job_descript_tfidf_vocab, job_type_vocab):
        self.job_descript_tfidf_vocab = job_descript_tfidf_vocab
        self.job_type_vocab = job_type_vocab

    def vectorize(self, tfidf, vector_length=-1):
        """
        Args:
            title (str): the string of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized title (numpy.array)
        """
        indices = [self.job_descript_tfidf_vocab.begin_seq_index]
        indices.extend(self.job_descript_tfidf_vocab.lookup_token(token) 
                       for token in tfidf.split(" "))
        indices.append(self.job_descript_tfidf_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.job_descript_tfidf_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, tfidf_df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            tfidf_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the CNNTfidfVectorizer
        """
        job_type_vocab = Vocabulary()        
        for job_type in sorted(set(tfidf_df.job_type)):
            job_type_vocab.add_token(job_type)

        word_counts = Counter()
        for tfidf in tfidf_df.tfidf:
            for token in tfidf.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1
        
        job_descript_tfidf_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                job_descript_tfidf_vocab.add_token(word)
        
        return cls(job_descript_tfidf_vocab, job_type_vocab)

    @classmethod
    def from_serializable(cls, contents):
        job_descript_tfidf_vocab = \
            SequenceVocabulary.from_serializable(contents['job_descript_tfidf_vocab'])
        job_type_vocab =  \
            Vocabulary.from_serializable(contents['job_type_vocab'])

        return cls(title_vocab=job_descript_tfidf_vocab, category_vocab=job_type_vocab)

    def to_serializable(self):
        return {'job_descript_tfidf_vocab': self.job_descript_tfidf_vocab.to_serializable(),
                'job_type_vocab': self.job_type_vocab.to_serializable()}

In [97]:
class CNNTfidfDataset(Dataset):
    def __init__(self, tfidf_df, vectorizer):
        """
        Args:
            tfidf_df (pandas.DataFrame): the dataset
            vectorizer (CNNTfidfVectorizer): vectorizer instatiated from dataset
        """
        self.tfidf_df = tfidf_df
        self._vectorizer = vectorizer

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, tfidf_df.tfidf)) + 2
        

        self.train_df = self.tfidf_df[self.tfidf_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.tfidf_df[self.tfidf_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.tfidf_df[self.tfidf_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Class weights
        class_counts = tfidf_df.job_type.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.job_type_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)
        
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, tfidf_df):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            tfidf_df (str): location of the dataset
        Returns:
            an instance of CNNTfidfDataset
        """
        tfidf_df = pd.read_csv(tfidf_df)
        train_tfidf_df = tfidf_df[tfidf_df.split=='train']
        return cls(tfidf_df, CNNTfidfVectorizer.from_dataframe(train_tfidf_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, tfidf_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            tfidf_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of CNNTfidfDataset
        """
        tfidf_csv = pd.read_csv(tfidf_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(tfidf_csv, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of CNNTfidfVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return CNNTfidfVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        job_descript_vector = \
            self._vectorizer.vectorize(row.tfidf, self._max_seq_length)

        job_type_index = \
            self._vectorizer.job_type_vocab.lookup_token(row.job_type)

        return {'x_data': job_descript_vector,
                'y_target': job_type_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [98]:
dataset = CNNTfidfDataset.load_dataset_and_make_vectorizer(args.output)
#dataset.save_vectorizer(args.vectorizer_file)
dataloader = DataLoader(dataset=dataset, batch_size=2,
                            shuffle=True, drop_last=True)

In [99]:
dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                batch_size=args.batch_size, 
                                device=args.device)

In [100]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [101]:
def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    with open(glove_filepath, encoding="utf8") as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings
    
def make_embedding_matrix_domain_speicific(embedding_filepath, words):
    """
    Create embedding matrix for a specific set of words.

    Args:
        embedding_filepath (str): file path to the word2vec embeddings
        words (list): list of words in the dataset
    """

    domain_embeddings = Word2Vec.load(embedding_filepath)
    word_to_idx = domain_embeddings.wv.key_to_index
    domain_vector = domain_embeddings.wv.get_normed_vectors()
    embedding_size = domain_vector.shape[1]

    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = domain_vector[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [102]:
if args.reload_from_files:
    # training from a checkpoint
    dataset = CNNTfidfDataset.load_dataset_and_load_vectorizer(args.output,
                                                           args.vectorizer_file)
else:
    # create dataset and vectorizer
    dataset = CNNTfidfDataset.load_dataset_and_make_vectorizer(args.output)
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove == True and args.use_domain_specific == False:
    words = vectorizer.job_descript_tfidf_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")

elif args.use_domain_specific == True and args.use_glove == False:
    words = vectorizer.job_descript_tfidf_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix_domain_speicific(embedding_filepath=args.domain_model_dir, 
                                       words=words)
    print("Using domain specific embeddings")

else:
    embeddings = None
    print("Using none embeddings")

classifier = CNNClassifier(embedding_size=args.embedding_size, 
                            num_embeddings=len(vectorizer.job_descript_tfidf_vocab),
                            num_channels=args.num_channels,
                            hidden_dim=args.hidden_dim, 
                            num_classes=len(vectorizer.job_type_vocab), 
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)

Using domain specific embeddings


In [ ]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
print("Test loss: {}".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 0.6523079535235531
Test Accuracy: 58.83152173913043


We tried the CNN with three different embeddings, none, pretrained and domain specific and the domain specific gave the highest accuracy using learning_rate=0.001, dropout_p=0.1, batch_size=128, early_stopping_criteria=5, embedding_size=100, hidden_dim=100, num_channels=100, and kernel_size=3 with the following results:

None embeddings:

Test loss: 0.6523476489212201

Test Accuracy: 58.86548913043478

Pretrained Embedding:

Test loss: 0.6516470131666766

Test Accuracy: 58.74660326086957

Domain Specific Embedding:

Test loss: 0.651664490285127

Test Accuracy: 58.93342391304348

Then we tried different parameters and try to get better accuracy, we run each test three times and find the average.

parameters kept: batch_size=128, embedding_size=100, early_stopping_criteria=5, kernel_size=3

| learning_rate | dropout_p | hidden_dim | num_channel | kernel_size | loss               | Accuracy           |
|---------------|-----------|------------|-------------|-------------|--------------------|--------------------|
| 0.001         | 0.1       | 100        | 100         | 3           | 0.651664490285127  | 58.93342391304348  |
| 0.01          | 0.1       | 100        | 100         | 3           | 0.6539218062939854 | 58.271059782608695 |
| 0.0001        | 0.1       | 100        | 100         | 3           | 0.6522674068160679 | 58.882472826086946 |
| 0.001         | 0.01      | 100        | 100         | 3           | 0.6520292020362357 | 58.729619565217384 |
| 0.001         | 0.3       | 100        | 100         | 3           | 0.6536173535429914 | 58.816929347826088 |
| 0.001         | 0.4       | 100        | 100         | 3           | 0.6538983091064121 | 58.86548913043479  |
| 0.001         | 0.6       | 100        | 100         | 3           | 0.6561724284420842 | 58.83152173913044  |
| 0.001         | 0.4       | 150        | 100         | 3           | 0.6529110907644466 | 58.72395833333333  |
| 0.001         | 0.4       | 150        | 100         | 3           | 0.6529110907644466 | 58.72395833333333  |
| 0.001         | 0.4       | 300        | 300         | 4           | 0.6522181824497554 | 58.89945652173913  |

As shown above changing those parameter does not improve the accuracy by much



In [ ]:
def predict_category(tfidf, classifier, vectorizer, max_length):
    """Predict a News category for a new title
    
    Args:
        title (str): a raw title string
        classifier (CNNtfidfClassifier): an instance of the trained classifier
        vectorizer (CNNTfidfVectorizer): the corresponding vectorizer
        max_length (int): the max sequence length
            Note: CNNs are sensitive to the input data tensor size. 
                  This ensures to keep it the same size as the training data
    """
    tfidf = preprocess_text(tfidf)
    vectorized_title = \
        torch.tensor(vectorizer.vectorize(tfidf, vector_length=max_length))
    result = classifier(vectorized_title.unsqueeze(0), apply_softmax=True)
    probability_values, indices = result.max(dim=1)
    predict_job_type = vectorizer.job_type_vocab.lookup_index(indices.item())

    return {'job_type': predict_job_type, 
            'probability': probability_values.item()}

In [ ]:
def get_samples():
    samples = {}
    for cat in dataset.val_df.job_type.unique():
        samples[cat] = dataset.val_df.tfidf[dataset.val_df.job_type==cat].tolist()[:5]
    return samples

val_samples = get_samples()

In [ ]:
classifier = classifier.to("cpu")

for truth, sample_group in val_samples.items():
    print(f"True Category: {truth}")
    print("="*30)
    for sample in sample_group:
        prediction = predict_category(sample, classifier, 
                                      vectorizer, dataset._max_seq_length + 1)
        print("Prediction: {} (p={:0.2f})".format(prediction['job_type'],
                                                  prediction['probability']))
        print("\t + Sample: {}".format(sample))
    print("-"*30 + "\n")

True Category: Full Time
Prediction: Other (p=0.57)
	 + Sample: kbs kaplan students business student domestic counselling accounting helping international
Prediction: Full Time (p=0.61)
	 + Sample: mammoet wind australian advisor quality renewables turbines assurance turbine farm
Prediction: Full Time (p=0.61)
	 + Sample: mammoet wind australian advisor quality renewables turbines assurance turbine farm
Prediction: Full Time (p=0.79)
	 + Sample: startrack collate freight analyst post unrivalled range logistics customers country
Prediction: Full Time (p=0.73)
	 + Sample: amp fm facilities leadership forecasting contract compliance corporate performance financial
------------------------------

True Category: Other
Prediction: Full Time (p=0.62)
	 + Sample: automation schneider electric place top process work management segreatpeople lifeison
Prediction: Full Time (p=0.51)
	 + Sample: pest anticimex flick control australias hygiene although qualificationscredentials onteamwork 35branches

The code above shows the true category from the predicted for each job type and in this case there are two job type such as full time and other. The p values specifies the confidence probability that it is true on the prediction and the probablities above are not high.

### CNN Conv1d based model using the job description

The main objective of this section is to build a cnn model to classify the job type binary classification using the job description collumn which should provide better classification as it has more unique data sets which can make it easier to seperate them into the types.

To swap between the word embedding set the different args in the following cells:

onehotencoder: use_glove=False, use_domain_specific=False,

pretrained: use_glove=True, use_domain_specific=False

domain_specific: use_glove=False, use_domain_specific=True

The models are saved in the specified folder which are shown in the cell below.

The model parameters used below will follows with the previous cnn model.

In [ ]:
args = Namespace(
    # Data and Path hyper parameters
    data_set="cleaned_data_new.csv",
    vectorizer_file="cnn_vectorizer.json",
    model_state_file="cnn_model.pth",
    output="output.csv",
    save_dir_descript_one_hot="model_storage\cnn_descript_onehot",
    save_dir_descript_pretrained="model_storage\cnn_descript_pretrained",
    save_dir_descript_domain_specific="model_storage\cnn_descript_domain_specific",
    domain_model_dir="model_storage\domain_model\domain.model",
    domain_file_name="domain.model",
    # Model hyper parameters
    glove_filepath='glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    use_domain_specific=True,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    kernel_size=3,
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=100, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    train_proportion=0.7,
    val_proportion=0.1,
    test_proportion=0.2
) 

In [ ]:
if args.use_glove == True:
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir_descript_pretrained,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir_descript_pretrained,
                                            args.model_state_file)

        # Check CUDA
        if not torch.cuda.is_available():
            args.cuda = False

        args.device = torch.device("cuda" if args.cuda else "cpu")
            
        print("Using CUDA: {}".format(args.cuda))


        # Set seed for reproducibility
        set_seed_everywhere(args.seed, args.cuda)

        # handle dirs
        handle_dirs(args.save_dir_descript_pretrained)

if args.use_domain_specific == True:
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir_descript_domain_specific,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir_descript_domain_specific,
                                            args.model_state_file)

        # Check CUDA
        if not torch.cuda.is_available():
            args.cuda = False

        args.device = torch.device("cuda" if args.cuda else "cpu")
            
        print("Using CUDA: {}".format(args.cuda))


        # Set seed for reproducibility
        set_seed_everywhere(args.seed, args.cuda)

        # handle dirs
        handle_dirs(args.save_dir_descript_domain_specific)

if args.use_glove == False and args.use_domain_specific == False:
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir_descript_one_hot,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir_descript_one_hot,
                                            args.model_state_file)
    

        # Check CUDA
        if not torch.cuda.is_available():
            args.cuda = False

        args.device = torch.device("cuda" if args.cuda else "cpu")
            
        print("Using CUDA: {}".format(args.cuda))


        # Set seed for reproducibility
        set_seed_everywhere(args.seed, args.cuda)

        # handle dirs
        handle_dirs(args.save_dir_descript_one_hot)

Using CUDA: True


In [ ]:
class CNNDescriptVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, job_descript_vocab, job_type_vocab):
        self.job_descript_vocab = job_descript_vocab
        self.job_type_vocab = job_type_vocab

    def vectorize(self, job_description_clean, vector_length=-1):
        """
        Args:
            job_description_clean (str): the string of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized job_description_clean (numpy.array)
        """
        indices = [self.job_descript_vocab.begin_seq_index]
        indices.extend(self.job_descript_vocab.lookup_token(token) 
                       for token in job_description_clean.split(" "))
        indices.append(self.job_descript_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.job_descript_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, job_descript_df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            job_descript_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the CNNDescriptVectorizer
        """
        job_type_vocab = Vocabulary()        
        for job_type in sorted(set(job_descript_df.job_type)):
            job_type_vocab.add_token(job_type)

        word_counts = Counter()
        for job_description_clean in job_descript_df.job_description_clean:
            for token in job_description_clean.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1
        
        job_descript_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                job_descript_vocab.add_token(word)
        
        return cls(job_descript_vocab, job_type_vocab)

    @classmethod
    def from_serializable(cls, contents):
        job_descript_vocab = \
            SequenceVocabulary.from_serializable(contents['job_descript_vocab'])
        job_type_vocab =  \
            Vocabulary.from_serializable(contents['job_type_vocab'])

        return cls(title_vocab=job_descript_vocab, category_vocab=job_type_vocab)

    def to_serializable(self):
        return {'job_descript_vocab': self.job_descript_vocab.to_serializable(),
                'job_type_vocab': self.job_type_vocab.to_serializable()}

In [ ]:
class CNNDescriptDataset(Dataset):
    def __init__(self, job_descript_df, vectorizer):
        """
        Args:
            job_descript_df (pandas.DataFrame): the dataset
            vectorizer (CNNDescriptVectorizer): vectorizer instatiated from dataset
        """
        self.job_descript_df = job_descript_df
        self._vectorizer = vectorizer

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, job_descript_df.job_description_clean)) + 2
        

        self.train_df = self.job_descript_df[self.job_descript_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.job_descript_df[self.job_descript_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.job_descript_df[self.job_descript_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Class weights
        class_counts = job_descript_df.job_type.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.job_type_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)
        
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, job_descript_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            job_descript_csv (str): location of the dataset
        Returns:
            an instance of CNNDescriptDataset
        """
        job_descript_df = pd.read_csv(job_descript_csv)
        train_job_descript_df = job_descript_df[job_descript_df.split=='train']
        return cls(job_descript_df, CNNDescriptVectorizer.from_dataframe(train_job_descript_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, job_descript_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            job_descript_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of CNNDescriptDataset
        """
        job_descript_csv = pd.read_csv(job_descript_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(job_descript_csv, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of CNNDescriptVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return CNNDescriptVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        job_descript_vector = \
            self._vectorizer.vectorize(row.job_description_clean, self._max_seq_length)

        job_type_index = \
            self._vectorizer.job_type_vocab.lookup_token(row.job_type)

        return {'x_data': job_descript_vector,
                'y_target': job_type_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [ ]:
dataset = CNNDescriptDataset.load_dataset_and_make_vectorizer(args.output)
#dataset.save_vectorizer(args.vectorizer_file)
dataloader = DataLoader(dataset=dataset, batch_size=2,
                            shuffle=True, drop_last=True)

In [ ]:
dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                batch_size=args.batch_size, 
                                device=args.device)

In [ ]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns: a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [ ]:
if args.reload_from_files:
    # training from a checkpoint
    dataset = CNNDescriptDataset.load_dataset_and_load_vectorizer(args.output,
                                                           args.vectorizer_file)
else:
    # create dataset and vectorizer
    dataset = CNNDescriptDataset.load_dataset_and_make_vectorizer(args.output)
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.job_descript_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")

elif args.use_domain_specific:
    words = vectorizer.job_descript_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix_domain_speicific(embedding_filepath=args.domain_model_dir, 
                                       words=words)
    print("Using domain specific embeddings")

else:
    embeddings = None
    print("Using none embeddings")

classifier = CNNClassifier(embedding_size=args.embedding_size, 
                            num_embeddings=len(vectorizer.job_descript_vocab),
                            num_channels=args.num_channels,
                            hidden_dim=args.hidden_dim, 
                            num_classes=len(vectorizer.job_type_vocab), 
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)

Using domain specific embeddings


In [ ]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 0.705501051376695;
Test Accuracy: 82.08220108695653


None vectoriser:

Test loss: 0.7476352446753047

Test Accuracy: 82.42187499999999

Pretrained Embedding:

Test loss: 0.5494730355946914

Test Accuracy: 82.33695652173911

Domain Specific Embedding:

Test loss: 0.705501051376695

Test Accuracy: 82.08220108695653

This shows that using the whole description is better to classify than using the tfidf words.

In [ ]:
def predict_category(job_description_clean, classifier, vectorizer, max_length):
    """Predict a job category for a new title
    
    Args:
        title (str): a raw title string
        classifier (CNNClassifier): an instance of the trained classifier
        vectorizer (CNNDescriptVectorizer): the corresponding vectorizer
        max_length (int): the max sequence length
            Note: CNNs are sensitive to the input data tensor size. 
                  This ensures to keep it the same size as the training data
    """
    job_description_clean = preprocess_text(job_description_clean)
    vectorized_title = \
        torch.tensor(vectorizer.vectorize(job_description_clean, vector_length=max_length))
    result = classifier(vectorized_title.unsqueeze(0), apply_softmax=True)
    probability_values, indices = result.max(dim=1)
    predict_job_type = vectorizer.job_type_vocab.lookup_index(indices.item())

    return {'job_type': predict_job_type, 
            'probability': probability_values.item()}

In [ ]:
def get_samples():
    samples = {}
    for cat in dataset.val_df.job_type.unique():
        samples[cat] = dataset.val_df.job_description_clean[dataset.val_df.job_type==cat].tolist()[:5]
    return samples

val_samples = get_samples()

In [ ]:
classifier = classifier.to("cpu")

for truth, sample_group in val_samples.items():
    print(f"True Category: {truth}")
    print("="*30)
    for sample in sample_group:
        prediction = predict_category(sample, classifier, 
                                      vectorizer, dataset._max_seq_length + 1)
        print("Prediction: {} (p={:0.2f})".format(prediction['job_type'],
                                                  prediction['probability']))
        print("\t + Sample: {}".format(sample))
    print("-"*30 + "\n")

True Category: Full Time
Prediction: Full Time (p=0.54)
	 + Sample:  located in western sydney our client is looking for an early childhood teacher that is energetic and dedicated to their role as a teacher their ideal teacher will be innovative and willing to deliver a responsive service that is committed to inclusive practices that support families cultural socioeconomic and individual situations what do you need to apply bachelor of early childhood teaching qualification acecqa approved nsw working with children check teacher accreditation number nesa approved child protection training first aid asthma and anaphylaxis certificate why apply for this role this particular organisation is recognised for their professionalism and high quality of care the organisation is committed to meeting the needs of their employees this includes having competitive salaries and opportunity for continuous development 
Prediction: Full Time (p=0.99)
	 + Sample:  on off you should hold a degree or equiva

The prediction above shows promising result as most of the probability are above 0.9 as compared to just using the tfidf as expected since they each have unique descriptions.